In [50]:
import pandas as pd

In [20]:
from table_evaluator import TableEvaluator

In [55]:
variables = ['sex','age','agegr','placesize','edu','socprof','marital','ls','depress','trust','trustfam','trustneigh','sport','nofriend','smoke','alcabuse','alcsol','wkabint','englang','height','weight','bmi']
original = pd.read_csv('df_original.csv', names = variables)
sample = pd.read_csv('df1_sample.csv', names = variables)
cart = pd.read_csv('df1_cart.csv', names = variables)
ctree = pd.read_csv('df1_ctree.csv', names = variables)
parametric = pd.read_csv('df1_parametric.csv', names = variables)
rf = pd.read_csv('df1_rf.csv', names = variables)
bag = pd.read_csv('df1_bag.csv', names = variables)
ctgan = pd.read_csv('ctgan.csv', names = variables)

FileNotFoundError: [Errno 2] No such file or directory: 'ctgan.csv'

In [27]:
original.head()

,sex,age,agegr,placesize,edu,socprof,marital,ls,depress,trust,...,sport,nofriend,smoke,alcabuse,alcsol,wkabint,englang,height,weight,bmi
0,FEMALE,57,45-59,"URBAN 100,000-200,000",VOCATIONAL/GRAMMAR,RETIRED,MARRIED,PLEASED,6,ONE CAN`T BE TOO CAREFUL,...,NO,6,NO,NO,NO,NO,NONE,170,89,30.7958
1,MALE,20,16-24,RURAL AREAS,VOCATIONAL/GRAMMAR,PUPIL OR STUDENT,SINGLE,MOSTLY SATISFIED,0,IT`S DIFFICULT TO TELL,...,NO,4,NO,NO,NO,NO,ACTIVE,187,82,23.4493
2,FEMALE,18,16-24,"URBAN 500,000 AND OVER",VOCATIONAL/GRAMMAR,PUPIL OR STUDENT,SINGLE,PLEASED,0,MOST PEOPLE CAN BE TRUSTED,...,NO,20,NO,NO,NO,NO,ACTIVE,165,50,18.3655
3,FEMALE,78,65+,RURAL AREAS,PRIMARY/NO EDUCATION,RETIRED,WIDOWED,MIXED,16,ONE CAN`T BE TOO CAREFUL,...,YES,0,NO,NO,NO,NO,NONE,160,78,30.4687
4,FEMALE,54,45-59,"URBAN 100,000-200,000",VOCATIONAL/GRAMMAR,SELF-EMPLOYED,MARRIED,MOSTLY SATISFIED,4,ONE CAN`T BE TOO CAREFUL,...,YES,6,YES,NO,NO,NO,NONE,158,50,20.0288


In [28]:
fac = ['sex','agegr','placesize','edu','socprof','marital','ls','trust','trustfam','trustneigh','sport','smoke','alcabuse','alcsol','wkabint','englang']

In [34]:
evaluator_cart = TableEvaluator(original, cart, cat_cols = fac)
evaluator_ctree = TableEvaluator(original, ctree, cat_cols = fac)
evaluator_parametric = TableEvaluator(original, parametric, cat_cols = fac)
evaluator_rf = TableEvaluator(original, rf, cat_cols = fac)
evaluator_bag = TableEvaluator(original, bag, cat_cols = fac)
evaluator_sample = TableEvaluator(original, sample, cat_cols = fac)

In [ ]:
#evaluator_cart.visual_evaluation()
#evaluator_ctree.visual_evaluation()
#evaluator_parametric.visual_evaluation()
#evaluator_rf.visual_evaluation()
#evaluator_bag.visual_evaluation()
graphique = evaluator_sample.visual_evaluation()

Les deux premiers graphiques représentent les logarithmes absolu de la moyenne et de l'écart-type de chaque variables numériques.

Matrice corrélations :
- Continue - Continue : utilise le coefficient de corrélation de Pearson $\frac{\sum(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum(x_i - \bar{x})^2\sum(y_i -\bar{y})^2}} \in [-1,1]$
- Continue - Catégorielle : utilise le rapport de corrélation $\eta^2(y,x) = \frac{SCE_{inter}}{SCE_{intra}} \in [0,1]$ avec $SCE_{intra} = \sum_j\sum_{i \in J_j}(y_{ij} - \bar{y}_{.j})^2$ qui est la variabilité intra-classe et $SCE_{inter} = \sum_j\sum_{i \in J_j}(\bar{y}_{.j} - \bar{y})^2$ qui est la variabilité inter-classe
- Catégorielle - Catégorielle : utilise le U de Theil, une métrique de corrélation asymétrique pour les associations catégorielles défini par $U(X|Y) = \frac{H(X) - H(X|Y)}{H(X)} \in [0,1]$ avec $H(X) = -\sum_x P_X(x)\log P_X(x)$ et $H(X|Y) = -\sum_{x,y}P_{X,Y}(x,y)\log P_{X|Y}(x|y)$

Cart : 
- Les moyennes et écarts types des variables numériques sont très proches après transformation logarithmique.
- Les corrélations entre variables sont bien reproduites dans les données synthétiques.
- La structure globale des données, telle que capturée par les deux premières composantes principales, est conservée dans les données synthétiques.

Ctree : Pareil

Parametric : 
- On constate une différence dans la variabilité des variables numériques entre les données originales et les données synthétiques.
- On apperçoit également dans la matrice des différences des cases plus foncées indiquant une plus grande différence dans les corrélations
- Les individus ont une représentation différente dans le premier plan factoriel
- Le modèle parametric est globalement moins bon que cart et ctree.

Rf :
- Plus grandes différences dans les effectifs de chaque variables par rapport aux autres modèles
- Il y a une assez grande différence dans les corrélations concernant la variable depress

Bag :
- Quelques différences au niveau des effectifs de cretaines variables

Sample :
- Est assez bon jusqu'à ce qu'on arrive aux corrélations qui sont catastrophiques tout comme la représentation dans le premier plan factoriel 

In [42]:
evaluator.evaluate(target_col = 'sex')

/opt/mamba/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/mamba/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.